In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from scipy.spatial.distance import cdist
from scipy.io import savemat
from mpl_toolkits.mplot3d import Axes3D

In [2]:
# Load data
POSITION = loadmat('data.mat')['data']
# Define the features and target variables
data_matrix = np.array(POSITION)

In [3]:
# Extract x, y, z columns
xyz_columns = data_matrix[:, :3]

# Calculate the center (mean) of the data
center = np.mean(xyz_columns, axis=0)

# Calculate distances from each point to the center
distances_to_center = np.linalg.norm(xyz_columns - center, axis=1)

# Create a list to store zones
zones = []

# Initialize a mask to track points assigned to zones
points_assigned_to_zones = np.zeros(len(data_matrix), dtype=bool)

# Iterate through each zone and extract data
for i in range(5):
    # Calculate the radius for the current zone
    current_radius = (i + 1) * (np.max(distances_to_center) / 5)

    # Create a mask for points within the sphere and not assigned to previous zones
    sphere_mask = np.logical_and(distances_to_center <= current_radius, ~points_assigned_to_zones)

    # Extract data for the current sphere
    sphere_data = data_matrix[sphere_mask]

    # Update the mask for points assigned to zones
    points_assigned_to_zones |= sphere_mask

    # Append the sphere data to the list
    zones.append(sphere_data)

    # Calculate x, y, z limits for the current zone
    x_limits = (np.min(sphere_data[:, 0]), np.max(sphere_data[:, 0]))
    y_limits = (np.min(sphere_data[:, 1]), np.max(sphere_data[:, 1]))
    z_limits = (np.min(sphere_data[:, 2]), np.max(sphere_data[:, 2]))

    # Print the limits for the current zone
    print(f"Zone {i + 1} Limits:")
    print(f"X Limits: {x_limits}")
    print(f"Y Limits: {y_limits}")
    print(f"Z Limits: {z_limits}")
    print()

# Print the center of the data
print(f"Center of the data: {center}")

# Print the zones
for i, zone in enumerate(zones):
    print(f"Zone {i + 1} has {len(zone)} points")

    # Save the sphere data to a .mat file (adjust the file name as needed)
    savemat(f'zone{i + 1}_centered.mat', {'zone_data': sphere_data})

Zone 1 Limits:
X Limits: (-75.75154516916541, 222.87057120668777)
Y Limits: (29.788632282441398, 269.20249368675593)
Z Limits: (200.87945889237403, 498.18922873507216)

Zone 2 Limits:
X Limits: (-248.2793742761516, 380.73827017255445)
Y Limits: (-174.2570721435739, 426.811859948895)
Z Limits: (45.9328278749935, 658.4631920891682)

Zone 3 Limits:
X Limits: (-406.7001180962947, 538.1676379036758)
Y Limits: (-357.499344767236, 578.6489588938718)
Z Limits: (-69.79931096456491, 701.1096154247078)

Zone 4 Limits:
X Limits: (-561.4112647425295, 611.2385013347721)
Y Limits: (-518.0278336153582, 613.134194069278)
Z Limits: (-69.79931096456491, 657.4721306098145)

Zone 5 Limits:
X Limits: (-611.2385013347721, 610.4515824817876)
Y Limits: (-613.134194069278, 525.4412029370488)
Z Limits: (-69.79931096456491, 422.3879778018695)

Center of the data: [ 65.70892016 111.90865023 343.44929414]
Zone 1 has 63150 points
Zone 2 has 1533775 points
Zone 3 has 1973515 points
Zone 4 has 1161054 points
Zone 5 ha

In [4]:
# Extract x, y, z columns
xyz_columns = data_matrix[:, :3]

# Calculate the center (mean) of the data
center = np.mean(xyz_columns, axis=0)

# Calculate distances from each point to the center
distances_to_center = np.linalg.norm(xyz_columns - center, axis=1)

# Initialize a mask to track points assigned to zones
points_assigned_to_zones = np.zeros(len(data_matrix), dtype=bool)

# Create a separate figure for each zone
for i in range(5):
    # Calculate the radius for the current zone
    current_radius = (i + 1) * (np.max(distances_to_center) / 5)

    # Create a mask for points within the sphere and not assigned to previous zones
    sphere_mask = np.logical_and(distances_to_center <= current_radius, ~points_assigned_to_zones)

    # Extract data for the current sphere
    sphere_data = data_matrix[sphere_mask]

    # Update the mask for points assigned to zones
    points_assigned_to_zones[sphere_mask] = True



In [5]:
import numpy as np
from tensorflow import keras

# Load the neural network models
models = [keras.models.load_model(f'model_z{i}.h5') for i in range(1, 6)]

def get_zone(x, y, z, roll, pitch, yaw):
    # Combine the input values into a 6-dimensional vector
    point_vector = np.array([x, y, z, roll, pitch, yaw])

    # Calculate the distance of the point to the center
    distance_to_center = np.linalg.norm(point_vector[:3] - center)

    # Determine the zone based on the distance to the center
    for i in range(5):
        current_radius = (i + 1) * (np.max(distances_to_center) / 5)
        if distance_to_center <= current_radius:
            # Use the neural network model for the current zone
            model = models[i]

            # Reshape the input point to match the model's input shape
            input_point = np.array(point_vector).reshape(1, -1)

            # Make predictions using the loaded model
            prediction = model.predict(input_point)*np.pi

            # Print the neural network prediction (adjust as needed)
            print(f"Zone {i + 1} - Model: model_z{i + 1}.h5")
            print(f"Neural network prediction for Zone {i + 1}: {prediction}")

            return i + 1

    # If the point is outside all zones, return 0 or handle accordingly
    return 0

resulting_zone = get_zone(350.000, 320.000, 200.000, 90.000, 0.000, 0.000)



1/1 [==============================] - 3s 3s/step
Zone 3 - Model: model_z3.h5
Neural network prediction for Zone 3: [[ 0.14054982 -0.32432693 -0.5349133   1.5601836   1.3285582   0.49878907]]


In [9]:
# Load data
squarefile = loadmat('square.mat')['square']
# Define the features and target variables
Square = np.array(squarefile)
print(Square)

[[350.         320.         200.          90.           0.
    0.        ]
 [350.         320.04685548 199.95314452  90.           0.
    0.        ]
 [350.         320.18742191 199.81257809  90.           0.
    0.        ]
 ...
 [350.         319.81257809 199.81257809  90.           0.
    0.        ]
 [350.         319.95314452 199.95314452  90.           0.
    0.        ]
 [350.         320.         200.          90.           0.
    0.        ]]


In [12]:
import numpy as np
from tensorflow import keras

# Load the neural network models
models = [keras.models.load_model(f'model_z{i}.h5') for i in range(1, 6)]

def get_zone(x, y, z, roll, pitch, yaw):
    # Combine the input values into a 6-dimensional vector
    point_vector = np.array([x, y, z, roll, pitch, yaw])

    # Calculate the distance of the point to the center
    distance_to_center = np.linalg.norm(point_vector[:3] - center)
    print(distance_to_center)
    # Determine the zone based on the distance to the center
    for i in range(5):
        current_radius = (i + 1) * (np.max(distances_to_center) / 5)
        if distance_to_center <= current_radius:
            # Use the neural network model for the current zone
            model = models[i]

            # Reshape the input point to match the model's input shape
            input_point = np.array(point_vector).reshape(1, -1)

            # Make predictions using the loaded model
            prediction = model.predict(input_point)*np.pi

            # Print the neural network prediction (adjust as needed)
            print(f"Zone {i + 1} - Model: model_z{i + 1}.h5")
            print(f"Neural network prediction for Zone {i + 1}: {prediction}")

            return i + 1

    # If the point is outside all zones, return 0 or handle accordingly
    return 0

resulting_zone = get_zone()

TypeError: get_zone() missing 6 required positional arguments: 'x', 'y', 'z', 'roll', 'pitch', and 'yaw'

In [18]:
import numpy as np
from tensorflow import keras
import scipy.io

# Load the square.mat file
squarefile = scipy.io.loadmat('square.mat')['square']

# Convert the data to a NumPy array
Square = np.array(squarefile)

# Extract x, y, z columns
xyz_columns = data_matrix[:, :3]

# Calculate the center (mean) of the data
center = np.mean(xyz_columns, axis=0)

# Calculate distances from each point to the center
distances_to_center = np.linalg.norm(xyz_columns - center, axis=1)

# Load the neural network models (update paths if needed)
models = [keras.models.load_model(f'model_z{i}.h5') for i in range(1, 6)]

# Function to calculate zone and predictions
def get_zone(points):
    predictions = []

    for point_vector in points:
        # Separate positional and angular data
        x, y, z, roll, pitch, yaw = point_vector

        # Calculate the distance of the point to the center
        distance_to_center = np.linalg.norm(np.array([x, y, z]) - center)

        # Determine the zone based on the distance to the center
        for i in range(5):
            if distance_to_center <= distances_to_center[i]:
                # Use the neural network model for the current zone
                model = models[i]

                # Reshape the input point to match the model's input shape
                input_point = np.array(point_vector).reshape(1, -1)

                # Make predictions using the loaded model
                prediction = model.predict(input_point) * np.pi

                # Append the original data and the prediction as a 7th column
                predictions.append([
                    x, y, z, roll, pitch, yaw, prediction[0, 0]  # Assuming prediction output is scalar
                ])
                break

    return np.array(predictions)

# Run the predictions
predicted_data = get_zone(Square)

# Save the predicted data to a new .mat file
scipy.io.savemat('predictions_with_zones.mat', {'predicted_data': predicted_data})

print("Predictions saved to 'predictions_with_zones.mat'")


1/1 [==============================] - 0s 24ms/step
Predictions saved to 'predictions_with_zones.mat'


In [24]:
import numpy as np
from tensorflow import keras
import scipy.io

# Load the square.mat file
squarefile = scipy.io.loadmat('square.mat')['square']

# Convert the data to a NumPy array
Square = np.array(squarefile)

# Calculate the center as the mean of x, y, and z coordinates
center = np.mean(Square[:, :3], axis=0)

# Calculate distances of all points to the center
distances = np.linalg.norm(Square[:, :3] - center, axis=1)

# Get the maximum distance to define the largest zone
max_distance = np.max(distances)

# Divide into 5 equal zones
distances_to_center = np.linspace(0, max_distance, 5)

# Load the neural network models (update paths if needed)
models = [keras.models.load_model(f'model_z{i}.h5') for i in range(1, 6)]

# Function to calculate zone and predictions
def get_zone(points):
    predictions = []

    for point_vector in points:
        # Separate positional and angular data
        x, y, z, roll, pitch, yaw = point_vector

        # Calculate the distance of the point to the center
        distance_to_center = np.linalg.norm(np.array([x, y, z]) - center)

        # Determine the zone based on the distance to the center
        for i in range(5):
            current_radius = (i + 1) * (np.max(distances_to_center) / 5)
            if distance_to_center <= current_radius:
                # Use the neural network model for the current zone
                model = models[i]

                # Reshape the input point to match the model's input shape
                input_point = np.array(point_vector).reshape(1, -1)

                # Make predictions using the loaded model
                prediction = model.predict(input_point) * np.pi

                # Append the original data and the prediction as a 7th column
                predictions.append([
                    x, y, z, roll, pitch, yaw, prediction[0, 0]
                ])

                # Break the loop once the zone is determined
                break

    return np.array(predictions)

# Run the predictions
predicted_data = get_zone(Square)

# Save the predicted data to a new .mat file
scipy.io.savemat('predictions_with_zones.mat', {'predicted_data': predicted_data})

print("Predictions saved to 'predictions_with_zones.mat'")


1/1 [==============================] - 0s 27ms/step
Predictions saved to 'predictions_with_zones.mat'
